In [27]:
import re
from pathlib import Path

import ipywidgets as widgets
from ipywidgets import Layout, Button, HBox, VBox, Box, Dropdown, Select, Text, Output, IntSlider, Label
from IPython.display import display, clear_output, Image, Video

import pandas as pd
import numpy as np

In [55]:
outputs_root = Path('images_out')
folder_prefix = 'permutations_limited_palette_2D'
folders = list(outputs_root.glob(f'{folder_prefix}_*'))
len(folders)

def format_val(v):
    try:
        v = float(v)
        if int(v) == v:
            v = int(v)
    except:
        pass
    return v

def parse_folder_name(folder):
    #chunks = folder.name[1+len(folder_prefix):].split('_')
    #chunks = folder.name[1+len(folder_prefix):].split('-')
    metadata_string = folder.name[1+len(folder_prefix):]
    pattern = r"_?([a-zA-Z_]+)-([0-9.]+)"
    matches = re.findall(pattern, metadata_string)
    d_ = {k:format_val(v) for k,v in matches}
    d_['fpath'] = folder
    d_['n_images'] = len(list(folder.glob('*.png')))
    return d_

#parse_folder_name(folders[0])
df_meta = pd.DataFrame([parse_folder_name(f) for f in folders])

variant_names = [v for v in df_meta.columns.tolist() if v not in ['fpath']]
variant_ranges = {v:df_meta[v].unique() for v in variant_names}
[v.sort() for v in variant_ranges.values()]
True

True

In [56]:
# to do: output and display palettes

kargs = {k:widgets.Dropdown(options=v, value=v[0], disabled=False, layout=Layout(width='auto')) for k,v in variant_ranges.items()}
kargs['i'] = widgets.IntSlider(min=1, max=40, step=1, value=1, continuous_update=False, readout=True, readout_format='d')

@widgets.interact(
    **kargs
)
def display_images(
    palettes,
    palette_size,
    gamma,
    hdr_weight,
    smoothing_weight,
    palette_normalization_weight,
    i,
):
    folder = df_meta[
        (palettes == df_meta['palettes']) &
        (palette_size == df_meta['palette_size']) &
        (gamma == df_meta['gamma']) &
        (hdr_weight == df_meta['hdr_weight']) &
        (smoothing_weight == df_meta['smoothing_weight']) &
        (palette_normalization_weight == df_meta['palette_normalization_weight'])
    ]['fpath'].values[0]
    im_path = str(folder / f"{folder.name}_{i}.png")
    return Image(im_path, width=700)


interactive(children=(Dropdown(description='palettes', layout=Layout(width='auto'), options=(10,), value=10), …